# __Predicting Outcomes of Call Option Contracts with Multi-Class Classification__

# SECTION 1: Preparation

##  1.1.) Loading the Python packages

In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path

# Load libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot
from pandas import read_csv, set_option
from pandas.plotting import scatter_matrix
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score, GridSearchCV

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.svm import SVC
#from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier

#importing classification_report, confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix

#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# import packages for hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

#from sklearn.metrics import balanced_accuracy_score
#from sklearn.metrics import confusion_matrix
#from imblearn.metrics import classification_report_imbalanced

#Libraries for Saving the Model
from pickle import dump
from pickle import load

import warnings
warnings.filterwarnings('ignore')



## 1.2.) Loading the Data

In [2]:
#Import call options info

df = pd.read_csv(
    Path("../Resources/spycallsV4.csv")
)


In [5]:
df.head(20)

,UNDERLYING_LAST,EXPIRE_DATE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,C_VOLUME,C_LAST,STRIKE,STRIKE_DISTANCE_PCT,STRIKE DISTANCE
0,477.47,1/7/2022,3.17,0.02300,0.01017,0.02686,-0.02841,0.00112,0.10755,945.0,0.05,488.0,0.022,10.53
1,477.47,1/7/2022,3.17,0.03244,0.01380,0.03645,-0.03910,0.00140,0.10587,929.0,0.07,487.0,0.020,9.53
2,477.47,1/7/2022,3.17,0.01355,0.00664,0.01750,-0.01714,0.00072,0.10644,616.0,0.03,489.0,0.024,11.53
3,467.69,1/7/2022,1.23,0.48150,0.06841,0.13640,-0.69786,0.01158,0.17024,4512.0,2.17,468.0,0.001,0.31
4,467.69,1/7/2022,1.23,0.41030,0.06925,0.13296,-0.64939,0.01054,0.16404,7511.0,1.67,469.0,0.003,1.31
5,477.47,1/7/2022,3.17,0.13568,0.04220,0.10827,-0.13578,0.00656,0.10471,2881.0,0.38,483.0,0.012,5.53
6,477.47,1/7/2022,3.17,0.18459,0.05108,0.13224,-0.17190,0.00877,0.10577,4546.0,0.53,482.0,0.009,4.53
7,477.47,1/7/2022,3.17,0.09724,0.03349,0.08570,-0.10330,0.00477,0.10417,2277.0,0.22,484.0,0.014,6.53
8,477.47,1/7/2022,3.17,0.04522,0.01875,0.04795,-0.05286,0.00194,0.10333,907.0,0.10,486.0,0.018,8.53
9,477.47,1/7/2022,3.17,0.06824,0.02570,0.06625,-0.07637,0.00351,0.10429,3151.0,0.15,485.0,0.016,7.53


In [ ]:
# Review the data types associated with the columns

df.dtypes

In [ ]:
#Check for any null values and remove the null values

print('Null Values =', dataset.isnull().values.any())

#Drop NaNs

In [ ]:
#Import further features

In [ ]:
#Check for any null values and remove the null values
print('Null Values =', dataset.isnull().values.any())

#Drop NaNs


In [ ]:
#Filter options data for contracts that have a strike price within 5% from spot price

In [ ]:
#Concatinate

# SECTION 2: Exploratory Data Analysis

## 2.1.) Descriptive Statistics

In [ ]:
df.shape

In [ ]:
display(df.head())
df.tail()

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
#Calculate the predicted variable, 'y', which is the perecent profit or loss

In [ ]:
#Filter 'y' into categories, according to the following:
#{0 = 'Strong Sell', 1 = 'Sell', 2 = 'Pass', 3 = 'Buy', 4 = 'Strong Buy'}
#{0: < -50%,
#1: > -50% to -20%,
#2: > -20% to 20%,
#3: > 20% to 50%,
#4: > 50%}

In [ ]:
#Count number in each category of 'y'

class_names = {0:'Strong Sell', 1:'Sell', 2:'Pass', 3:'Buy', 4:'Strong Buy'}
print(df.Class.value_counts().rename(index = class_names))

## 2.2.) Feature Analysis and Exploration

### Plotting features according to contract length

In [4]:
#
contract_outcome = dataset.groupby('''insert var here''')['y'].value_counts(normalize=True).loc[:,1]
sns.set(rc={'figure.figsize':(12,5)})
sns.barplot(x=contract_outcome.index, y=contract_outcome.values, color='#5975A4', saturation=1)

NameError: name 'dataset' is not defined

### Eliminate Uncorrelated Features

In [ ]:
#Calculate correlation of each feature with 'y'

correlation = df.corr()
correlation_df = abs(correlation['y'])

In [ ]:
correlation_df.sort_values(ascending=False)

In [ ]:
#Drop variables with less than 3% correlation with contract outcome ('y')

drop_list_corr = sorted(list(correlation_df[correlation_df < 0.03].index))
print(drop_list_corr)

In [ ]:
df.drop(labels=drop_list_corr, axis=1, inplace=True)
df.shape

### Drop the 'ContractID' column from the DataFrame

In [ ]:
# Drop the 'ContractID' column from the DataFrame

df = applicant_data_df.drop(columns=["ContractID"])

# Review the DataFrame

df.head()

# SECTION 3: Evaluate Algorithms and Models

## 3.1.) Train Test Split and Evaluation Metrics

In [ ]:
#Seperate features from predicted variable, 'y', which is outcome of call options contracts

y = df["y"]

X = df.drop(columns='y')

In [ ]:
#Split into training and testing groups

validation_size = 0.2

seed = 7

X_train, X_validation, y_train, y_validation = train_test_split(X, y, stratify=y, test_size=validation_size, random_state=seed)

In [ ]:
#Normalize features data

# Create the StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Scale both the training and testing data from the features dataset
X_train_scaled = X_scaler.transform(X_train)
X_validation_scaled = X_scaler.transform(X_validation)

## 3.2.) Quick Check of Models and Algorithms, Select the Best

In [ ]:
#Spot check Classification algorithms

models = []

#models.append(('SVM', SVC()))

#Boosting methods

models.append(('XGB', XGBClassifier()))
models.append(('AB', AdaBoostClassifier()))
models.append(('GBM', GradientBoostingClassifier()))

#Bagging methods

models.append(('RF', RandomForestClassifier()))
models.append(('ET', ExtraTreesClassifier()))


In [ ]:
#Test options for classification

num_folds = 10
seed = 7

In [ ]:
results = []

names = []

scoring = 'accuracy'

for name, model in models:
    kfold = StratifiedKFold(n_splits=num_folds, random_state=seed, shuffle=True)
    cv_results = cross_val_score(model, X_train_scaled, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    findings_summary = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(findings_summary)

In [ ]:
#Compare the model outcomes

fig = pyplot.figure()
fig.suptitle('Model Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
fig.set_size_inches(8,4)
pyplot.show()

## 3.3.) Selecting Random Forest *(or whichever tree method best)*, Calculating its Baseline 

In [6]:
#Estimate accuracy on training set with cross validation

rf_model = RandomForestClassifier()

cv = StratifiedKFold(n_splits=num_folds, random_state=seed, shuffle=True)
scores = cross_val_score(rf_model, X_train_scaled, y_train, scoring='accuracy', cv=cv)
avg_score = np.mean(scores)
avg_score

NameError: name 'num_folds' is not defined

In [ ]:
#Estimate accuracy on validation set

rf_model = RandomForestClassifier()

rf_model.fit(X_train_scaled, y_train)

predictions = rf_model.predict(X_validation_scaled)

print(accuracy_score(y_validation, predictions))

print('Weighted Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_validation, predictions, average='weighted')))

print(classification_report(y_validation, predictions, target_names=['Strong Sell', 'Sell', 'Pass']))

print(confusion_matrix(y_validation, predictions))

In [ ]:
#Visualization of the Confusion Matrix

df_matrix = pd.DataFrame(confusion_matrix(y_validation, predictions), columns=np.unique(y_validation), index = np.unique(y_validation))
df_matrix.index.name = 'Actual'
df_matrix.columns.name = 'Predicted'
sns.heatmap(df_matrix, cmap="Blues", annot=True, annot_kws={"size": 16}) 

# SECTION 4: Model Tuning 

## 4.1.) Grid Search for (some random forest)

In [ ]:
# Grid Search: (select model) Tuning

n_estimators = [20, 100, 180, 1000]
max_features = [1, 2, 3, 4]

param_grid = dict(n_estimators=n_estimators, max_features=max_features)

rf_model = RandomForestClassifier()

kfold = StratifiedKFold(n_splits=num_folds, random_state=seed, shuffle=True)

grid = GridSearchCV(estimator=rf_model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(X_train_scaled, y_train)

#Print Results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
ranks = grid_result.cv_results_['rank_test_score']
for mean, stdev, param, rank in zip(means, stds, params, ranks):
    print("#%d %f (%f) with: %r" % (rank, mean, stdev, param))

In [ ]:
#Prepare the model with parameters decided in previous cell

rf_model = RandomForestClassifier(max_features=3, n_estimators=20)
rf_model.fit(X_train_scaled, y_train)

In [ ]:
#Score predictions of training set

training_predictions = rf_model.predict(X_train_scaled)

print(accuracy_score(y_validation, predictions))

print('Micro Precision: {:.2f}'.format(precision_score(y_train, training_predictions, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_train, training_predictions, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_train, training_predictions, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_train, training_predictions, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_train, training_predictions, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_train, training_predictions, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_train, training_predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_train, training_predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_train, training_predictions, average='weighted')))


print(classification_report(y_train, training_predictions, target_names=['Strong Sell', 'Sell', 'Pass']))

In [ ]:
#Score predictions of validation set

predictions = xgb_model.predict(X_validation_scaled)

print(accuracy_score(y_validation, predictions))

print('Micro Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_validation, predictions, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_validation, predictions, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_validation, predictions, average='weighted')))


print(classification_report(y_validation, predictions, target_names=['Strong Sell', 'Sell', 'Pass', 'Buy', 'Strong Buy']))

print(confusion_matrix(y_validation, predictions))

In [ ]:
#Visualization of the Confusion Matrix

df_matrix = pd.DataFrame(confusion_matrix(y_validation, predictions), columns=np.unique(y_validation), index = np.unique(y_validation))
df_matrix.index.name = 'Actual'
df_matrix.columns.name = 'Predicted'
sns.heatmap(df_matrix, cmap="Blues", annot=True, annot_kws={"size": 16}) 

## 4.2.) Determining Feature Importance

In [ ]:
#Feature importances

importances = rf_model.feature_importances_

In [ ]:
important_features = zip(X.columns, rf_model.feature_importances_)

In [ ]:
importances_df = pd.DataFrame(important_features)

In [ ]:
importances_df = importances_df.rename(columns={0: 'Feature', 1: 'Importance'})

In [ ]:
importances_df = importances_df.set_index('Feature')

In [ ]:
importances_df = importances_df.sort_values(by='Importance', ascending=False)

In [ ]:
importances_df

In [ ]:
#Plot the top 10 most important features

importances_df[0:10].plot(
    kind='barh',
    color='green',
    title='Feature Importance',
    legend=True)

## 4.3.) Bayesian Optimization with HYPEROPT for XGBoost

In [ ]:
#Initialize domain space for range of values 
 
space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1, 9),
        'reg_alpha' : hp.quniform('reg_alpha', 40, 180, 1),
        'reg_lambda' : hp.uniform('reg_lambda', 0, 1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5, 1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }


The available hyperopt optimization algorithms are -

hp.choice(label, options) — Returns one of the options, which should be a list or tuple.

hp.randint(label, upper) — Returns a random integer between the range [0, upper).

hp.uniform(label, low, high) — Returns a value uniformly between low and high.

hp.quniform(label, low, high, q) — Returns a value round(uniform(low, high) / q) * q, i.e it rounds the decimal values and returns an integer.

hp.normal(label, mean, std) — Returns a real value that’s normally-distributed with mean and standard deviation sigma.

In [ ]:
def objective(space):
    xgb_model = xgb.XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [(X_train_scaled, y_train), (X_validation_scaled, y_validation)]
    
    xgb_model.fit(X_train_scaled, y_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    

    pred = xgb_model.predict(X_validation_scaled)
    accuracy = accuracy_score(y_validation, pred>0.5)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

In [ ]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

In [ ]:
print("The best hyperparameters are : ","\n")
print(best_hyperparams)

In [ ]:
xgb_model = xgb.XGBClassifier(colsample_bytree=1, max_depth=6, gamma=6.879347321724768, min_child_weight=1, reg_lambda=1, reg_alpha=0)

xgb_model.fit(X_train_scaled, y_train)

In [ ]:
#Predicting on training set

training_predictions = xgb_model.predict(X_train_scaled)

print(accuracy_score(y_train, training_predictions))

print('Weighted Precision: {:.2f}'.format(precision_score(y_train, training_predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_train, training_predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_train, training_predictions, average='weighted')))

print(classification_report(y_train, training_predictions, target_names=['Strong Sell', 'Sell', 'Pass', 'Buy', 'Strong Buy']))

In [ ]:
#Predicting on validation set

predictions = xgb_model.predict(X_validation_scaled)

print(accuracy_score(y_validation, predictions))

print('Weighted Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_validation, predictions, average='weighted')))

print(classification_report(y_validation, predictions, target_names=['Strong Sell', 'Sell', 'Pass', 'Buy', 'Strong Buy']))

print(confusion_matrix(y_validation, predictions))

In [ ]:
#Get probabilities of predictions on validation set

proba_df = xgb_model.predict_proba(X_validation_scaled)
proba_df

In [ ]:
#Forming DataFrame with probabilities

proba_df = pd.DataFrame(y_proba.round(2))

In [ ]:
#Formatting probabilities DataFrame

proba_df["Actual"] = y_validation
proba_df.columns = ["'Strong Sell' Probability Estimate", "'Sell' Probability Estimate", "'Pass' Probability Estimate", "Actual", "'Buy' Probability Estimate", "'Strong Buy' Probability Estimate"]
proba_df

In [ ]:
#Labeling caterogires

proba_df.loc[proba_df["Actual"] == 0, "Actual"] = "Strong Sell"
proba_df.loc[proba_df["Actual"] == 1, "Actual"] = "Sell"
proba_df.loc[proba_df["Actual"] == 2, "Actual"] = "Pass"
proba_df.loc[proba_df["Actual"] == 1, "Actual"] = "Buy"
proba_df.loc[proba_df["Actual"] == 2, "Actual"] = "Strong Buy"

In [ ]:
#Get ROC AUC score

roc_auc_score(y_validation, pred_proba, multi_class="ovr")

## 4.4.) Determining Feature Importance for XGBoost Model

In [ ]:
#Feature importances

importances = rf_model.feature_importances_

In [ ]:
important_features = zip(X.columns, rf_model.feature_importances_)

In [ ]:
importances_df = pd.DataFrame(important_features)

In [ ]:
importances_df = importances_df.rename(columns={0: 'Feature', 1: 'Importance'})

In [ ]:
importances_df = importances_df.set_index('Feature')

In [ ]:
importances_df = importances_df.sort_values(by='Importance', ascending=False)

In [ ]:
importances_df

In [ ]:
#Plot the top 10 most important features

importances_df[0:10].plot(
    kind='barh',
    color='green',
    title='Feature Importance',
    legend=True)

In [ ]:
et_model = ExtraTreesClassifier()
et_model.fit(X_train_scaled, y_train)

In [ ]:
ab_model = AdaBoostClassifier()
ab_model.fit(X_train_scaled, y_train)

In [ ]:
gb_model = GradientBoostingClassifier()
ab_model.fit(X_train_scaled, y_train)